<a href="https://colab.research.google.com/github/j20nch/class2022Spring/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
import gradio as gr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 24.2 MB/s 
     |████████████████████████████████| 84 kB 2.3 MB/s 
     |████████████████████████████████| 54 kB 649 kB/s 
     |████████████████████████████████| 1.1 MB 46.8 MB/s 
     |████████████████████████████████| 212 kB 74.1 MB/s 
     |████████████████████████████████| 2.0 MB 59.7 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
     |████████████████████████████████| 256 kB 69.7 MB/s 
     |████████████████████████████████| 271 kB 76.7 MB/s 
     |████████████████████████████████| 144 kB 72.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 11.1 MB 62.3 MB/s 
     |████████████████████████████████| 80 kB 11.2 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 62 

gradio library는 공통적으로 깔아야 함! (데모 구현을 위해)

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 32.9 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
     |████████████████████████████████| 596 kB 72.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


링크를 타고 들어가면 Image classification에 대한 설명이 있음. 그 설명들도 이해한 걸 가정한 채 시험문제에 나올 수도 있으니 직접 웹페이지에 들어가서 잘 살펴보기.
Image classification은 입력으로 그림이 들어가서, 모델을 거치고 나와서는 다양한 종류(class)를 확률, 가능성과 함께 보여준다!

In [3]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


In [ ]:
<Script 형태>

1) 필요한 것들을 import

2) 우리가 하려 하는 task는 image를 classify하는 것. 고로 image 하나가 sample로 들어와야 classify를 할 수 있을 것임. 하나의 url 주소가 있고 .jpg가 있는 걸 봐선 그림/사진 링크임을 유추할 수 있음. image 변수 안에는 그림 데이터가 어떤 식으로든지 숫자 형태로 저장되어 있구나! 

3) pretrained: "행렬". 행렬은 곧 AI 모델. 데이터로부터 잘 학습된/훈련된/돈의 가치가 있는 그런 모델을 가지고 오는 것. 
Feature extraction을 할 수 있는 모델임을 변수 이름으로부터 추측 가능. 아랫줄은 Extracted된 Feature 정보를 가지고 classify를 하는 모델이구나!

*Feature는 뭔가? 2)의 image는 그냥 숫자 정보임. 어떤 흑백 사진이 있으면 밝으면 1, 흐리면 0. 이런 식의 숫자 정보 (칼라면 3층(R, G, B 정보), 흑백이면 1층) 그 정보를 다 쓰기에는 너무나 많은 정보임. (ex: 사람을 classify한다? 그 사람에 대한 모든 정보를 가진다면 거의 무한대의 정보가 있게 됨. 대신 feature(object를 정의할 수 있는 제한된 수의 정보)의 형태로 뽑는다면, 머리색, 키, 몸무게, 성별, 눈매, 학교...등. 그런 걸 해주는 게 feature_extractor.

4) inputs: image정보로부터 그 feature 정보를 뽑아냄. (훨씬 개수가 줄어들었을 것)
outputs: classify된 정보가 나옴.
> predicted_Class_idx. class의 idx가 잡힘 (총 1000개의 class가 이미 ImageNet에 들어 있음. 그것 중에 하나를 predict. 위에서 입력한 feature 정보로부터 1000개의 class가 준비가 되어 있는 것. 그 1000개에 대한 확률정보가 나옴(1000개의 숫자가 도출될 것이고, 그걸 다 더하면 1이 될 것)
그래서 가장 확률이 높은 것(argmax)을 print해준다! ("Predicted class: ~~" 구조로)

In [4]:
def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

<function 형태>

입력부분: image. url이 입력이 아니라, image가 입력이 되는 것. 
출력부분: model.config.id2label[predicted_class_idx]

> 이 함수는 image(그림 파일이 숫자 형태로 바꿔진) 정보가 들어와서, class 중 하나가 출력

In [5]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

0

예시 사진을 데모에 넣어주는 것. 강아지와 호랑이를 url로 받고, os.system("curl " + url + " > tiger.jpg")의 의미: 위에 있는 호랑이 사진 파일을 tiger.jpg 이름으로 해서, 현재 구글 계정에, class 계정에 있는 두 파일을 복사해서 갖다놓으라는 의미

In [6]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://35185.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fcbe2dcf8d0>,
 'http://127.0.0.1:7860/',
 'https://35185.gradio.app')

input과 output 유형을 정하고(function의 입출력 유형과 맞추면 됨), examples에 tiger.jpg, dog.jpg(아까 이름 넣은 거) 쓰면 됨.
